In [ ]:
import json
import random
import pandas as pd
import numpy as np

# 1. 데이터 로드
with open('서울_전처리_번역_20개.json', 'r', encoding='utf-8') as f:
    teams = json.load(f)

# 2. 키워드-수치 점수 매핑 (사용자 말을 Score로 변환하는 다리 역할)
score_map = {
    "strength": ["강한", "실력 있는", "우승 후보", "압도적인"],
    "money": ["자본력이 좋은", "돈 많은", "투자를 많이 하는"],
    "star_power": ["스타 선수가 있는", "유명한", "화려한"],
    "attack_style": ["공격적인", "화끈한", "속도감 있는"],
    "underdog_feel": ["언더독", "약팀의 반란", "도전하는"],
    "fan_passion": ["팬덤이 뜨거운", "응원이 열정적인", "인기 많은"],
    "tradition": ["역사가 깊은", "전통 있는", "근본 있는", "명문"]
}

def generate_soft_variance_data(num = 1000):
    dataset = []
    unique_tags = list(set([tag for t in teams for tag in t['style_tags']]))

    for _ in range(num):
        # 랜덤하게 1~2개의 수치 카테고리와 1개의 태그 선택
        selected_cats = random.sample(list(score_map.keys()), random.randint(1, 2))
        selected_tag = random.sample(unique_tags, 1)[0]
        
        # 자연스러운 한글 질문 생성
        keywords = [random.choice(score_map[cat]) for cat in selected_cats]
        query = f"나는 수비가 좋고 골을 많이 넣는 팀이 좋아!!"
        
        for team in teams:
            # ① S_semantic (태그 매치 - 비중 낮춤): 0.4 가중치
            tag_match = 1.0 if selected_tag in team['style_tags'] else 0.2
            
            # ② S_relational (수치 점수 매치 - 비중 높임): 0.6 가중치
            # 선택된 카테고리들의 점수 평균을 내서 반영
            vibe_scores = [team['scores'][cat] / 10.0 for cat in selected_cats]
            s_relational = np.mean(vibe_scores)
            
            # ③ W_id (가중치 필터)
            w_id = 0.9 + (team['scores']['underdog_feel'] / 100) + (team['scores']['tradition'] / 100)
            
            # 최종 점수 계산: (태그*0.4 + 수치*0.6) * 가중치
            final_score = (0.4 * tag_match + 0.6 * s_relational) * w_id
            
            dataset.append({
                "user_query": query,
                "team_name": team['team_name'],
                "label_score": round(min(0.98, final_score), 4),
                "focused_scores": ", ".join(selected_cats)
            })
            
    return pd.DataFrame(dataset)

# 실행 및 저장
df_nuanced = generate_soft_variance_data(300) # 300번 반복 (팀당 300개 질문)
df_nuanced.to_csv("nuanced_score_data20.csv", index = False, encoding='utf-8-sig')